In [17]:
import pandas as pd
import glob
import itertools


In [18]:
# importa tutti i pickle delle statistiche sui duelli e salvali in un dizionario

VERSION_NUMBER = '4'
path = r'C:\Users\Carlo\0-dev\AOT_simulator\data\v'+VERSION_NUMBER+'\stats' # use your path
all_files = glob.glob(path + "/*.pkl")

li = []

for filename in all_files:
    df = pd.read_pickle(filename)
    li.append(df)

dict_duels = {(x.index[0],x.index[1]):x for x in li}

In [19]:
# crea una lista ordinata di tutti i giganti

titans_list = []
for k,v in dict_duels.items():
    if k[0] not in titans_list:
        titans_list.append(k[0])
    if k[1] not in titans_list:
        titans_list.append(k[1])
titans_list.sort()

In [20]:
# aggiungi al dizionario dei duelli il win_rate 0.5 di ogni gigante contro sé stesso

for titan in titans_list:
    dict_duels[(titan,titan)] = pd.DataFrame(data={'win_rate':[50,50]}) 

In [21]:
# salva tutte le possibili combinazioni di duelli in una lista

duels_names = [x for x in itertools.combinations(titans_list, 2)]
duels_names

[('Armoured Titan', 'Attack Titan'),
 ('Armoured Titan', 'Beast Titan'),
 ('Armoured Titan', 'Cart Titan'),
 ('Armoured Titan', 'Colossal Titan'),
 ('Armoured Titan', 'Female Titan'),
 ('Armoured Titan', 'Jaw Titan'),
 ('Armoured Titan', 'Warhammer Titan'),
 ('Attack Titan', 'Beast Titan'),
 ('Attack Titan', 'Cart Titan'),
 ('Attack Titan', 'Colossal Titan'),
 ('Attack Titan', 'Female Titan'),
 ('Attack Titan', 'Jaw Titan'),
 ('Attack Titan', 'Warhammer Titan'),
 ('Beast Titan', 'Cart Titan'),
 ('Beast Titan', 'Colossal Titan'),
 ('Beast Titan', 'Female Titan'),
 ('Beast Titan', 'Jaw Titan'),
 ('Beast Titan', 'Warhammer Titan'),
 ('Cart Titan', 'Colossal Titan'),
 ('Cart Titan', 'Female Titan'),
 ('Cart Titan', 'Jaw Titan'),
 ('Cart Titan', 'Warhammer Titan'),
 ('Colossal Titan', 'Female Titan'),
 ('Colossal Titan', 'Jaw Titan'),
 ('Colossal Titan', 'Warhammer Titan'),
 ('Female Titan', 'Jaw Titan'),
 ('Female Titan', 'Warhammer Titan'),
 ('Jaw Titan', 'Warhammer Titan')]

In [22]:
# crea un dizionario che associa ad ogni gigante la lista ordinata di tutti i suoi duelli

dict_duels_names = {}
for titan in titans_list:
    duels = [x for x in duels_names if titan in x]
    duels.append((titan,titan))
    duels = sorted(duels, key=lambda tup: (tup[0],tup[1]) )
    dict_duels_names[titan] = duels
    

In [23]:
# genera il dataframe con i win_rate di tutti i duelli

d = {}
for titan, duels in dict_duels_names.items():
    win_rates = []
    for duel in duels:
        if (duel[0], duel[1]) in dict_duels:
            pos = (duel[0], duel[1]).index(titan)
            win_rates.append(dict_duels[(duel[0], duel[1])]['win_rate'][pos])
        elif (duel[1], duel[0]) in dict_duels:
            pos = (duel[1], duel[0]).index(titan)
            win_rates.append(dict_duels[(duel[1], duel[0])]['win_rate'][pos]) 
    d[titan] = win_rates

In [25]:
win_ratio_matrix = pd.DataFrame(data=d, index=titans_list)
heatmap = win_ratio_matrix.T.style.background_gradient(axis=1)
heatmap

,Armoured Titan,Attack Titan,Beast Titan,Cart Titan,Colossal Titan,Female Titan,Jaw Titan,Warhammer Titan
Armoured Titan,50.000000,55.820000,99.990000,98.770000,75.340000,53.110000,63.680000,45.310000
Attack Titan,44.180000,50.000000,97.090000,95.940000,58.890000,50.280000,53.950000,43.810000
Beast Titan,0.010000,2.910000,50.000000,57.680000,2.500000,4.000000,5.980000,3.230000
Cart Titan,1.230000,4.060000,42.320000,50.000000,2.200000,5.310000,7.930000,4.250000
Colossal Titan,24.660000,41.110000,97.500000,97.800000,50.000000,49.890000,47.600000,27.800000
Female Titan,46.890000,49.720000,96.000000,94.690000,50.110000,50.000000,52.780000,45.800000
Jaw Titan,36.320000,46.050000,94.020000,92.070000,52.400000,47.220000,50.000000,42.810000
Warhammer Titan,54.690000,56.190000,96.770000,95.750000,72.200000,54.200000,57.190000,50.000000


In [26]:
# Salva la heatmap in un file html
html_str = heatmap.render()
Html_file= open('output_html\\v'+VERSION_NUMBER+'\\heatmap\\'+'win_rate_heatmap_v'+VERSION_NUMBER+'.html','w')
Html_file.write(html_str)
Html_file.close()

In [27]:
for k,v in dict_duels.items():
    html_str = v.to_html()
    Html_file= open('output_html\\v'+VERSION_NUMBER+'\\'+k[0].replace(" ", "")+ k[1].replace(" ", "") +'stats_v'+VERSION_NUMBER+'.html',"w")
    Html_file.write(html_str)
    Html_file.close()